22.11.4661 - Agus Heryanto

#1. Data Collection

In [55]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from flask import Flask, jsonify, request

In [56]:
# 2. Data Collection
data_csv = "https://raw.githubusercontent.com/agusheryanto182/job-recommendations/refs/heads/develop/scrapper/linkedin_jobs_20250407_020136.csv"
jobs_data = pd.read_csv(data_csv)
df = pd.DataFrame(jobs_data)
df.head()

,job_id,title,company,location_city,location_province,location_country,seniority_level,employment_level,job_function,industries,posted_date,description,keywords,link,num_applicants
0,4199554439,Internship Web Developer,PT. Sasa Inti,Jakarta,Jakarta,Indonesia,internship,internship,engineering and information technology,manufacturing,2025-04-03,DESKRIPSI PEKERJAANMendukung pengembangan situ...,web,https://id.linkedin.com/jobs/view/internship-w...,Over 200 applicants
1,4188450252,Front End Developer,PT Adicipta Inovasi Teknologi,Jakarta,Jakarta,Indonesia,associate,full-time,engineering and information technology,information technology & services,2025-03-24,Develop and maintain web applications using Re...,api|architect|architecture|bachelor|backend|co...,https://id.linkedin.com/jobs/view/front-end-de...,26 applicants
2,4178221580,Web Developer,PT Bank Digital BCA (BCA Digital),South Jakarta,Jakarta,Indonesia,associate,full-time,information technology,banking,2025-03-17,"ResponsibilitiesAnalyse, design and develop an...",architect|architecture|collaboration|communica...,https://id.linkedin.com/jobs/view/web-develope...,Over 200 applicants
3,4158977188,Web Developer,PT Astra International Tbk,Jakarta,Jakarta,Indonesia,entry level,full-time,engineering and information technology,automation machinery manufacturing,2025-03-08,Deskripsi PekerjaanMengembangkan aplikasi berb...,java|javascript|php|web,https://id.linkedin.com/jobs/view/web-develope...,Over 200 applicants
4,4170847328,Frontend Developer (React JS),Padepokan Tujuh Sembilan,Bandung,West Java,Indonesia,entry level,contract,engineering and information technology,information technology & services,2025-03-08,ResponsibilitiesMengembangkan dan memelihara a...,api|architect|architecture|aws|backend|ci/cd|c...,https://id.linkedin.com/jobs/view/frontend-dev...,70 applicants


#2. Preprocessing
Konsep: Membersihkan dan mengubah data mentah menjadi format yang bisa diproses model.

Langkah:

- Handling missing values

- Normalisasi

In [57]:
#3. Preprocessing
def preprocess_data(df):
    # make a copy dataframe
    df = df.copy()

    # remove the column that not used
    columns_to_drop = [
        'location_city',
        'location_province',
        'location_country',
        'employment_level',
        'job_function',
        'industries',
        ]

    # 1. Handling missing values
    df['keywords'] = df['keywords'].str.replace('|', ', ')
    df['title'] = df['title'].fillna('')
    df['seniority_level'] = df['seniority_level'].fillna('')
    df = df.drop(columns=columns_to_drop)

    # 3. Menggabungkan fitur teks yang relevan untuk content-based filtering
    df['combined_features'] = (
        df['title'] + ' ' +
        df['keywords'] + ' ' +
        df['seniority_level']
    ).str.lower()  # Mengubah ke lowercase

    # Handle np.nan in 'combined_features' by replacing them with empty strings
    df['combined_features'] = df['combined_features'].fillna('')

    return df

# Contoh penggunaan
processed_df = preprocess_data(df.copy())

# Melihat hasil preprocessing
print("\nSample hasil preprocessing:")
print("\nCombined Features (first row):")
print(processed_df['combined_features'].iloc[2][:10000])  # Menampilkan 200 karakter pertama


Sample hasil preprocessing:

Combined Features (first row):
web developer architect, architecture, collaboration, communication, design, development, docker, documentation, frontend, git, infrastructure, integration, java, kubernetes, next.js, operations, react, redux, security, spring, support, system integration, systems, testing, troubleshooting, typescript, web associate


#3. Model Building
Konsep: Membangun model rekomendasi menggunakan algoritma Content-Based Filtering (CBF)

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

class JobRecommender:
    def __init__(self):
        self.tfidf = TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 3),  # Tambahkan trigram untuk menangkap frasa yang lebih panjang
            max_features=10000,  # Tingkatkan jumlah fitur
            min_df=2,
            max_df=0.85,  # Turunkan max_df untuk menghilangkan kata yang terlalu umum
            analyzer='word',
            token_pattern=r'(?u)\b\w+[a-zA-Z.-]+\w*\b'   # Tangkap kata tunggal
        )
        self.tfidf_matrix = None
        self.df = None

    def fit(self, df):
        self.df = df
        self.tfidf_matrix = self.tfidf.fit_transform(df['combined_features'])

    def preprocess_text(self, text):
        """
        Preprocessing text dengan penanganan khusus untuk keyword teknologi

        Args:
            text: Teks yang akan diproses

        Returns:
            Teks yang sudah diproses
        """
        # Normalisasi variasi penulisan teknologi sebelum preprocessing umum
        tech_variations = {
            'javascript': ['js', 'javascript', 'ecmascript'],
            'typescript': ['ts', 'typescript'],
            'frontend': ['front-end', 'front end', 'frontend'],
            'backend': ['back-end', 'back end', 'backend'],
            'fullstack': ['full-stack', 'full stack', 'fullstack'],
            'reactjs': ['react', 'react.js', 'reactjs'],
            'nodejs': ['node.js', 'node js', 'nodejs'],
            'vuejs': ['vue', 'vue.js', 'vuejs'],
            'postgresql': ['postgres', 'postgresql'],
            'mongodb': ['mongo', 'mongodb'],
            'golang': ['go', 'golang'],
            'python': ['py', 'python'],
            'devops': ['dev ops', 'devops'],
            'machine learning': ['ml', 'machine learning'],
            'artificial intelligence': ['ai', 'artificial intelligence']
        }

        # Lowercase
        text = text.lower().strip()

        # Normalisasi teknologi
        for main_tech, variations in tech_variations.items():
            for variant in variations:
                text = text.replace(variant, main_tech)

        # Hapus karakter khusus tapi pertahankan yang penting
        text = re.sub(r'[^a-zA-Z\s,+#.]', '', text)

        # Normalisasi spasi
        text = ' '.join(word.strip() for word in text.split())

        return text

    def get_recommendations_from_cv(self, cv_text, n_recommendations=50):
        try:
            # Preprocessing CV
            processed_cv = self.preprocess_text(cv_text)

            # Transform CV text to vektor TF-IDF
            cv_vector = self.tfidf.transform([processed_cv])

            # Calculate cosine similarity
            cosine_sim = cosine_similarity(cv_vector, self.tfidf_matrix)[0]

            # Filter and sort based on similarity score
            job_scores = list(enumerate(cosine_sim))
            job_scores = sorted(job_scores, key=lambda x: x[1], reverse=True)

            # Take N top recommendation
            top_jobs = job_scores[:n_recommendations]

            if not top_jobs:
                return None

            # Prepare the result
            job_indices = [idx for idx, _ in top_jobs]
            recommendations = self.df.iloc[job_indices][[
                'job_id',
                'title',
                'company',
                'posted_date',
                'num_applicants',
                'link',
                'description',
                'keywords',
                'seniority_level'
            ]].copy()

            # add the score similarity
            recommendations['similarity_score'] = [score for _, score in top_jobs]
            recommendations['rank'] = range(1, len(recommendations) + 1)

            return recommendations

        except Exception as e:
            print(f"Error dalam mendapatkan rekomendasi: {str(e)}")
            return None

def print_recommendations(recommendations):
    """Menampilkan hasil rekomendasi dengan format yang lebih baik"""
    if recommendations is None:
        print("\nTidak ditemukan rekomendasi yang sesuai.")
        return

    print("\n=== Rekomendasi Pekerjaan Berdasarkan CV ===")
    print(f"Ditemukan {len(recommendations)} pekerjaan yang sesuai\n")

    for _, job in recommendations.iterrows():
        match_percentage = job['similarity_score'] * 100
        print(f"#{job['rank']} {job['title']}")
        print(f"Company: {job['company']}")
        print(f"Description: {job['description']}")
        print(f"Level: {job['seniority_level']}")
        print(f"Link: {job['link']}")
        print(f"Posted Date: {job['posted_date']}")
        print(f"Num Applicants: {job['num_applicants']}")
        print(f"Keywords dalam Job: {job['keywords']}")
        print("-" * 50)

# Penggunaan dengan jumlah rekomendasi yang lebih banyak
if __name__ == "__main__":
    # 1. Inisialisasi model
    recommender = JobRecommender()

    # 2. Fit model dengan dataset
    recommender.fit(processed_df)

    # 3. Contoh CV
    sample_cv = """
    Golang
    """

    # 4. Dapatkan rekomendasi menggunakan instance recommender
    recommendations = recommender.get_recommendations_from_cv(sample_cv, n_recommendations=50)

    # 5. Tampilkan hasil
    print_recommendations(recommendations)


=== Rekomendasi Pekerjaan Berdasarkan CV ===
Ditemukan 50 pekerjaan yang sesuai

#1 Internship Web Developer
Company: PT. Sasa Inti
Description: DESKRIPSI PEKERJAANMendukung pengembangan situs web berdasarkan kebutuhan bisnis.Mendukung pengembangan dan penyelesaian pengembangan web sesuai dengan jadwal yang telah ditetapkan.Membantu dalam penyusunan dokumen terkait pengembangan web (spesifikasi teknis, skenario dan hasil pengujian, serta panduan pengguna).
Level: internship
Link: https://id.linkedin.com/jobs/view/internship-web-developer-at-pt-sasa-inti-4199554439?trk=public_jobs_topcard-title
Posted Date: 2025-04-03
Num Applicants: Over 200 applicants
Keywords dalam Job: web
--------------------------------------------------
#2 Front End Developer
Company: PT Adicipta Inovasi Teknologi
Description: Develop and maintain web applications using React.js, including integration with backend APIs.Optimize application performance to be fast, responsive, and user-friendly.Contribute to archi

#4. Evaluation

In [61]:
class SimpleKeywordEvaluator:
    def evaluate_recommendations(self, cv_text: str, recommendations: pd.DataFrame) -> Dict:
        """
        Evaluasi sederhana: cek kemunculan setiap keyword CV di rekomendasi pekerjaan

        Args:
            cv_text: Text CV dengan keyword
            recommendations: DataFrame hasil rekomendasi
        """
        # Ambil keyword dari CV
        cv_keywords = [k.strip().lower() for k in cv_text.replace('\n', ' ').split(',')]

        print(f"\n=== Evaluasi Rekomendasi Pekerjaan ===")
        print(f"Keywords dalam CV: {', '.join(cv_keywords)}")
        print(f"Jumlah rekomendasi: {len(recommendations)}")
        print("\nAnalisis per Keyword:")

        # Analisis per keyword
        keyword_stats = {}
        for keyword in cv_keywords:
            # Hitung berapa rekomendasi yang mengandung keyword ini
            matching_jobs = 0
            matching_titles = []

            for _, job in recommendations.iterrows():
                # Cek di title dan keywords
                job_text = f"{str(job['title']).lower()} {str(job['keywords']).lower()}"
                if keyword in job_text:
                    matching_jobs += 1
                    matching_titles.append(job['title'])

            percentage = (matching_jobs / len(recommendations)) * 100
            keyword_stats[keyword] = {
                'matching_jobs': matching_jobs,
                'percentage': percentage,
                'matching_titles': matching_titles
            }

            print(f"\nKeyword: {keyword}")
            print(f"- Muncul di {matching_jobs} rekomendasi ({percentage:.1f}%)")
            print("- Pekerjaan yang cocok:")
            for title in matching_titles[:5]:  # Tampilkan 5 contoh
                print(f"  * {title}")

        # Hitung statistik keseluruhan
        total_matches = sum(stats['matching_jobs'] for stats in keyword_stats.values())
        avg_keyword_presence = total_matches / (len(cv_keywords) * len(recommendations)) * 100

        print("\n=== Ringkasan ===")
        print(f"Rata-rata kemunculan keyword: {avg_keyword_presence:.1f}%")

        return keyword_stats

# Contoh penggunaan
if __name__ == "__main__":
    # 1. Inisialisasi
    recommender = JobRecommender()
    recommender.fit(processed_df)
    evaluator = SimpleKeywordEvaluator()

    # 2. Test dengan CV
    cv_text = "Frontend, javascript, typescript, fullstack"

    # 3. Dapatkan rekomendasi
    recommendations = recommender.get_recommendations_from_cv(cv_text, n_recommendations=30)

    # 4. Evaluasi
    if recommendations is not None:
        evaluator.evaluate_recommendations(cv_text, recommendations)


=== Evaluasi Rekomendasi Pekerjaan ===
Keywords dalam CV: frontend, javascript, typescript, fullstack
Jumlah rekomendasi: 30

Analisis per Keyword:

Keyword: frontend
- Muncul di 27 rekomendasi (90.0%)
- Pekerjaan yang cocok:
  * Fullstack Developer
  * Senior Fullstack Engineer
  * Frontend Developer
  * JavaScript Developer Apprentice
  * Frontend Engineer - Brand Application (open to remote across ANZ)

Keyword: javascript
- Muncul di 22 rekomendasi (73.3%)
- Pekerjaan yang cocok:
  * Fullstack Developer
  * JavaScript Developer Apprentice
  * fullstack - Node and Angular / React Developer
  * Senior Software Engineer - Video Export (Fullstack)
  * Frontend Engineer - Brand Application (open to remote across ANZ)

Keyword: typescript
- Muncul di 23 rekomendasi (76.7%)
- Pekerjaan yang cocok:
  * Senior Fullstack Engineer
  * Frontend Developer
  * fullstack - Node and Angular / React Developer
  * Senior Software Engineer - Video Export (Fullstack)
  * Frontend Engineer - Brand App

#5. Deployment
Menyematkan model ke lingkungan produksi (API, aplikasi), simple dengan contoh Simpel API dengan Flask

In [60]:
app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def get_recommendations():
    """Endpoint untuk mendapatkan rekomendasi"""
    try:
        data = request.json
        cv_text = data['cv_text']
        n_recommendations = data.get('n_recommendations', 5)

        recommendations = recommender.get_recommendations_from_cv(
            cv_text, n_recommendations
        )

        if recommendations is None:
            return jsonify({
                'status': 'error',
                'message': 'Tidak dapat menghasilkan rekomendasi'
            })

        # Analisis keyword matching
        keyword_analysis = analyze_keywords(recommendations, cv_text)

        return jsonify({
            'status': 'success',
            'recommendations': recommendations.to_dict('records'),
            'keyword_analysis': keyword_analysis
        })

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        })

def print_recommendations(cv_text):
    """Function untuk print rekomendasi"""
    print(f"\nRekomendasi untuk CV dengan keyword: {cv_text}")
    recommendations = recommender.get_recommendations_from_cv(cv_text)

    if recommendations is None:
        print("Tidak dapat menghasilkan rekomendasi")
        return

    for _, job in recommendations.iterrows():
        print(f"\n#{job['rank']} {job['title']}")
        print(f"Level: {job['seniority_level']}")
        print(f"Keywords: {job['keywords']}")
        print("-" * 50)

if __name__ == '__main__':
    # Inisialisasi model
    recommender = JobRecommender()
    # Use the preprocessed DataFrame here instead of df
    recommender.fit(processed_df) # This line has changed

    # Contoh penggunaan
    test_cv = "Frontend, javascript, typescript, fullstack"
    print_recommendations(test_cv)

    # Jalankan API
    app.run(debug=True)


Rekomendasi untuk CV dengan keyword: Frontend, javascript, typescript, fullstack

#1 Fullstack Developer
Level: entry level
Keywords: api, backend, css, database, frontend, java, javascript, lua, mysql, next.js, node.js, react, spring, sql, web
--------------------------------------------------

#2 Senior Fullstack Engineer
Level: mid-senior level
Keywords: api, architect, architecture, backend, communication, communication skills, degree, deployment, design, development, engineering, expert, express, fintech, frontend, implementation, intern, ios, postgresql, product, react, security, sql, startup, technical architecture, typescript, web, web development
--------------------------------------------------

#3 Frontend Developer
Level: mid-senior level
Keywords: agile, backend, clean code, design, fast-paced, frontend, less, next.js, optimization, product, react, scala, typescript, web
--------------------------------------------------

#4 JavaScript Developer Apprentice
Level: interns

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
